Authors: Dror & Mariia

In [24]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, nltk, warnings
import matplotlib.cm as cm
import itertools
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
from wordcloud import WordCloud, STOPWORDS
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from IPython.display import display, HTML
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
%matplotlib inline

In [25]:
df = pd.read_csv('drive/My Drive/customer_segmentation.csv', encoding="ISO-8859-1") # from https://www.kaggle.com/carrie1/ecommerce-data
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## 1 First, let's see what data do we have

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [27]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [28]:
df.duplicated().sum()

5268

## 2 EDA

### 2.1. 25% of entries are null-valued customers. 

So, let's select this subgroup, save it separately and later check it. Should it be removed from our main data? 

Since we are clustering orders, these orders with undefined customers will be assigned to the same customer if we do some fillna - this is bad. Probably, we have to work with that data separately

In [29]:
null_customers = df[df.CustomerID.isna()]
null_customers.to_csv('drive/My Drive/null_customers.csv')  # later we will return to this
null_customers.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom


Now, we will deal with data without NaNs:

In [30]:
# a few options
df1 = df.dropna(axis = 0, subset = ['CustomerID'])
df2 = df[~df.CustomerID.isna()] 
np.all(df1 == df2)

True

### 2.2. Duplicates

In [31]:
df1.duplicated().sum()

5225

In [32]:
df = df1.drop_duplicates()

In [33]:
df.info()  # maybe reset index?

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401604 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    401604 non-null  object        
 1   StockCode    401604 non-null  object        
 2   Description  401604 non-null  object        
 3   Quantity     401604 non-null  int64         
 4   InvoiceDate  401604 non-null  datetime64[ns]
 5   UnitPrice    401604 non-null  float64       
 6   CustomerID   401604 non-null  float64       
 7   Country      401604 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.6+ MB


In [34]:
dfr = df.reset_index()

### 2.3. Transform 'InvoiceDate' into some number of separate features

In [35]:
df['year'] = pd.DatetimeIndex(df['InvoiceDate']).year
df['month'] = pd.DatetimeIndex(df['InvoiceDate']).month
df['dayofweek'] = pd.DatetimeIndex(df['InvoiceDate']).dayofweek
df['weekofyear'] = pd.DatetimeIndex(df['InvoiceDate']).weekofyear
df['hour'] = pd.DatetimeIndex(df['InvoiceDate']).hour
df['dayofyear'] = pd.DatetimeIndex(df['InvoiceDate']).dayofyear

# looks like we got all information from the InvoiceDate, so
df = df.drop('InvoiceDate', axis=1)

In [36]:
# let's see how many unique values for each column maybe do we have
for col in df.columns:
    print(f'Column {col} has {len(np.unique(df[col]))} unique values')

Column InvoiceNo has 22190 unique values
Column StockCode has 3684 unique values
Column Description has 3896 unique values
Column Quantity has 436 unique values
Column UnitPrice has 620 unique values
Column CustomerID has 4372 unique values
Column Country has 37 unique values
Column year has 2 unique values
Column month has 12 unique values
Column dayofweek has 6 unique values
Column weekofyear has 51 unique values
Column hour has 15 unique values
Column dayofyear has 298 unique values


Regarding 6 days in a week :) probably, the orders are just not processed on some day of the week and hence moved to the next day. we will see it further

In [56]:
df.dayofweek.value_counts()  # hmmm

3    81575
2    69753
1    67376
0    65715
6    61673
4    55512
Name: dayofweek, dtype: int64

### 2.4. Transform 'Country' column into: 1. binary column (UK or not),   2. onehotencoded

In [37]:
df['is_UK'] = df.Country.apply(lambda x: int(x == 'UK'))
df.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,year,month,dayofweek,weekofyear,hour,dayofyear,is_UK
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010,12,2,48,8,335,0
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010,12,2,48,8,335,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010,12,2,48,8,335,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010,12,2,48,8,335,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010,12,2,48,8,335,0


one hot encode countries

In [49]:
df_Countries = pd.get_dummies(df.Country)
df_Countries.head()

,Australia,Austria,Bahrain,Belgium,Brazil,Canada,Channel Islands,Cyprus,Czech Republic,Denmark,EIRE,European Community,Finland,France,Germany,Greece,Iceland,Israel,Italy,Japan,Lebanon,Lithuania,Malta,Netherlands,Norway,Poland,Portugal,RSA,Saudi Arabia,Singapore,Spain,Sweden,Switzerland,USA,United Arab Emirates,United Kingdom,Unspecified
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [55]:
df = pd.concat([df, df_Countries], axis=1)
df.head(2)

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,year,month,dayofweek,weekofyear,hour,dayofyear,is_UK,Australia,Austria,Bahrain,Belgium,Brazil,Canada,Channel Islands,Cyprus,Czech Republic,Denmark,EIRE,European Community,Finland,France,Germany,Greece,Iceland,Israel,Italy,Japan,Lebanon,Lithuania,Malta,Netherlands,Norway,Poland,Portugal,RSA,Saudi Arabia,Singapore,Spain,Sweden,Switzerland,USA,United Arab Emirates,United Kingdom,Unspecified
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010,12,2,48,8,335,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010,12,2,48,8,335,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


### Further steps

Later we will drop the column 'Country'. For now, we have to figure out what can we do with 'Description' and 'CustomerID'



As for Description, there are few options:


    - each value in this column should be turned into fixed length sequence of n-dimensional vectors, 
    where n - is either embedding dimension or size of vocabulary of most frequent words (this is CountVectorizer)

    - classify values in description column by item type or sth else (like stuff for kitchen/bedroom/etc)



As for CustomerID, 

    - I don't like that the values are integers, since there is no relations between these numbers, 
    in fact it's categorical feature, not numerical



In [61]:
for d in df.Description[:50]:
    print(d, end=' _____ ')

WHITE HANGING HEART T-LIGHT HOLDER _____ WHITE METAL LANTERN _____ CREAM CUPID HEARTS COAT HANGER _____ KNITTED UNION FLAG HOT WATER BOTTLE _____ RED WOOLLY HOTTIE WHITE HEART. _____ SET 7 BABUSHKA NESTING BOXES _____ GLASS STAR FROSTED T-LIGHT HOLDER _____ HAND WARMER UNION JACK _____ HAND WARMER RED POLKA DOT _____ ASSORTED COLOUR BIRD ORNAMENT _____ POPPY'S PLAYHOUSE BEDROOM  _____ POPPY'S PLAYHOUSE KITCHEN _____ FELTCRAFT PRINCESS CHARLOTTE DOLL _____ IVORY KNITTED MUG COSY  _____ BOX OF 6 ASSORTED COLOUR TEASPOONS _____ BOX OF VINTAGE JIGSAW BLOCKS  _____ BOX OF VINTAGE ALPHABET BLOCKS _____ HOME BUILDING BLOCK WORD _____ LOVE BUILDING BLOCK WORD _____ RECIPE BOX WITH METAL HEART _____ DOORMAT NEW ENGLAND _____ JAM MAKING SET WITH JARS _____ RED COAT RACK PARIS FASHION _____ YELLOW COAT RACK PARIS FASHION _____ BLUE COAT RACK PARIS FASHION _____ BATH BUILDING BLOCK WORD _____ ALARM CLOCK BAKELIKE PINK _____ ALARM CLOCK BAKELIKE RED  _____ ALARM CLOCK BAKELIKE GREEN _____ PANDA AND

In [63]:
df.dtypes

InvoiceNo                object
StockCode                object
Description              object
Quantity                  int64
UnitPrice               float64
CustomerID              float64
Country                  object
year                      int64
month                     int64
dayofweek                 int64
weekofyear                int64
hour                      int64
dayofyear                 int64
is_UK                     int64
Australia                 uint8
Austria                   uint8
Bahrain                   uint8
Belgium                   uint8
Brazil                    uint8
Canada                    uint8
Channel Islands           uint8
Cyprus                    uint8
Czech Republic            uint8
Denmark                   uint8
EIRE                      uint8
European Community        uint8
Finland                   uint8
France                    uint8
Germany                   uint8
Greece                    uint8
Iceland                   uint8
Israel  

Okay, possible plots:

    - plot boxplots for features with small number of unique values

    - for Country, week, dayofweek, dayofyear and month, etc we can aggregate total number of orders, total sum of orders made, etc

    - sth else?
